It's not possible for Pandas to store NaN values in integer columns.

This makes float the obvious default choice for data storage, because as soon as missing value arises Pandas would have to change the data type for the entire column. And missing values arise very often in practice.

As for why this is, it's a restriction inherited from Numpy. Basically, Pandas needs to set aside a particular bit pattern to represent NaN. This is straightforward for floating point numbers and it's defined in the IEEE 754 standard. It's more awkward and less efficient to do this for a fixed-width integer.

TODOS:
- X ---- Join delle tabelle
- Le colonne min_age, avg_age, max_age hanno degli int con valori fuori norma, rimuovere
- Le colonne 15,16,17 hanno un problema di tipi, investigare
- Usare unique per capire quali colonne hanno dati da ripulire (es. inc_char2: Suicide^)

In [43]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats import pearsonr

# To show all columns
pd.options.display.max_columns = None

# Add ID column to use as generic index
df = pd.read_csv('incidents.csv', header=0)
df['ID'] = range(1, len(df) + 1)
df.set_index('ID', inplace=True)

# Pretty-print
# display(df.head())

df_poverty = pd.read_csv('povertyByStateYear.csv')
df_district = pd.read_csv('year_state_district_house.csv')

# Add and reorder year column to join poverty table
columns = df.columns
df['year'] = pd.to_datetime(df['date']).dt.year
df = df[columns.insert(columns.get_loc('date') + 1, 'year')]
j_df = pd.merge(df, df_poverty, on=['year','state'], how='left')

# display(j_df.head())

# Workaround to join district table
temp = j_df['state']
j_df['state'] = j_df['state'].str.upper()
j2_df = pd.merge(j_df, df_district, on=['year','state','congressional_district'], how='left')
j2_df['state'] = temp

# WIP To reorder added tables
# columns = j2_df.columns
# j2_df = j2_df[columns.insert(columns.get_loc('congressional_district') + 1, ['party','candidatevotes','totalvotes'])]

display(j2_df.head())
j2_df.info()


/tmp/ipykernel_15743/592030983.py:15: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('incidents.csv', header=0)


,date,year,state,city_or_county,address,latitude,longitude,congressional_district,state_house_district,state_senate_district,participant_age1,participant_age_group1,participant_gender1,min_age_participants,avg_age_participants,max_age_participants,n_participants_child,n_participants_teen,n_participants_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,notes,incident_characteristics1,incident_characteristics2,povertyPercentage,party,candidatevotes,totalvotes
0,2015-05-02,2015,Indiana,Indianapolis,Lafayette Road and Pike Plaza,39.8322,-86.2492,7.0,94.0,33.0,19.0,Adult 18+,Male,19.0,19.0,19.0,0.0,0.0,1.0,1.0,0.0,0,1,0.0,0.0,1.0,Teen wounded while walking - Security guard at...,Shot - Wounded/Injured,NaN,12.3,NaN,NaN,NaN
1,2017-04-03,2017,Pennsylvania,Kane,5647 US 6,41.6645,-78.7856,5.0,NaN,NaN,62.0,Adult 18+,Male,62.0,62.0,62.0,0.0,0.0,1.0,1.0,0.0,1,0,0.0,0.0,1.0,shot self after accident,"Shot - Dead (murder, accidental, suicide)",Suicide^,10.5,NaN,NaN,NaN
2,2016-11-05,2016,Michigan,Detroit,6200 Block of East McNichols Road,42.4190,-83.0393,14.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0.0,1.0,2.0,1 inj.,Shot - Wounded/Injured,NaN,11.0,DEMOCRAT,244135.0,310974.0
3,2016-10-15,2016,District of Columbia,Washington,"1000 block of Bladensburg Road, NE",38.9030,-76.9820,1.0,NaN,NaN,NaN,Adult 18+,Male,248339,707477,761203,0.0,0.0,1.0,1.0,0.0,0,1,0.0,0.0,2.0,NaN,Shot - Wounded/Injured,NaN,14.9,NaN,NaN,NaN
4,2030-06-14,2030,Pennsylvania,Pittsburgh,California and Marshall Avenues,40.4621,-80.0308,14.0,NaN,NaN,NaN,Adult 18+,Male,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0,1,0.0,1.0,2.0,NaN,Shot - Wounded/Injured,"Drive-by (car to street, car to car)",NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239677 entries, 0 to 239676
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date                       239677 non-null  object 
 1   year                       239677 non-null  int32  
 2   state                      239677 non-null  object 
 3   city_or_county             239677 non-null  object 
 4   address                    223180 non-null  object 
 5   latitude                   231754 non-null  float64
 6   longitude                  231754 non-null  float64
 7   congressional_district     227733 non-null  float64
 8   state_house_district       200905 non-null  float64
 9   state_senate_district      207342 non-null  float64
 10  participant_age1           147379 non-null  float64
 11  participant_age_group1     197558 non-null  object 
 12  participant_gender1        203315 non-null  object 
 13  min_age_participants       16